# COGS 108 - Data Checkpoint

# Names

- Tong Zeng
- Kaiyu Guan
- Will Sumerfield
- Janty Sphabmixay

<a id='research_question'></a>
# Research Question

To what degree can we predict the crime rates in the counties of San Diego using data including population density, weather, temperature, income, and the number of police stations in these areas?

# Dataset(s)

- **Dataset Name:** Monthly Temperature in all counties in California, 2016
- **Link to the dataset:** https://www.ncdc.noaa.gov/cag/county/mapping/4/tavg/201902/1/value
- **Number of observations:** 696

This dataset contains the monthly information about the average temperature and precipitation of all 58 counties in
California in 2016. This dataset is obtained by merging two datasets(average temperature, average precipitation)
which are both accessible on NCDC.

- **Dataset Name:** Police Per County 2016
- **Link to the dataset:** https://github.com/COGS108/Group052-Fa21/blob/master/Datasets/Police/Police Per County 2016.csv
- **Number of observations:** 58

This dataset contains the number of policemen/women in the Counties of California in 2016.
We acquired this data by downloading directly from fbi.gov where this data is completely public for everyone to use.

- **Dataset Name:** Income Per County 2016
- **Link to the dataset:** https://data.census.gov/cedsci/table?q=California%20Income&t=Earnings%20%28Individuals%29&g=0400000US06%240500000&tid=ACSST1Y2019.S1902
- **Number of observations:** 40

This dataset contains the Mean Income of counties in California in USD per year, along with the margin of error for that data.
This data comes from the United States Census, and is available for public use.

- **Dataset Name:** Population Density Per County
- **Link to the dataset:** https://www.indexmundi.com/facts/united-states/quick-facts/california/population-density#table
- **Number of observations:** 58

This dataset contains the population density in people per square mile in each county, in 2016. This data comes from the California Government Data Portal called Open Justice.

- **Dataset Name:** Arrests Per County
- **Link to the dataset:** https://openjustice.doj.ca.gov/data
- **Number of observations:** 2,476

This dataset is still under processing. One group member (Janty) has been having technical issues with Jupyter Notebook,
and has received permission from Professor Fleischer to complete this data by checkpoint 2.

# Setup

In [1]:
import pandas as pd

# Data Cleaning

We first download the four datasets from the websites respectively.
Then we retrieve columns that contain our desired variables, rename those columns to be more comprehensive,
and check for null values.

<b>Weather</b>

In [2]:
#import precipitation dataset and temperature dataset
precip = pd.read_csv('Datasets/Weather/precip.csv')
temp = pd.read_csv('Datasets/Weather/temp.csv')

#Retrive data in 2016 only
temp = temp[temp['Date'].astype('str').str.contains('2016')]
precip = precip[precip['Date'].astype('str').str.contains('2016')]

#Retrive columns with useful information
temp = temp[['Location', 'Date', 'Value']]
precip = precip[['Location', 'Date', 'Value']]

#Rename columns
temp.columns = ['County', 'Month', 'Avg_Temperature']
precip.columns = ['County', 'Month', 'Avg_Precipitation']

#Merge the two datasets
weather = temp.merge(precip)

#Returns the month of the str
def find_month(date):
    return int(str(date)[-2:])

#Returns the county name from the given str
def county(coun):
    return coun.strip(' County')

#Apply the functions to find the month and county name
weather['Month'] = weather['Month'].apply(find_month)
weather['County'] = weather['County'].apply(county)

weather.head()

,County,Month,Avg_Temperature,Avg_Precipitation
0,Alameda,1,50.0,5.65
1,Alameda,2,55.2,0.84
2,Alameda,3,55.1,5.25
3,Alameda,4,59.1,1.94
4,Alameda,5,62.8,0.24


<b>Police</b>

In [3]:
#Read the police dataset
police = pd.read_csv('Datasets/Police Per County/Police Per County 2016.csv')

#Drop first four unnessary rows
police = police.drop(police.index[[0, 1, 2, 3]])

#Reindex the dataset
police = police.reset_index(drop = True)

#Drop an empty column
police = police.drop(['Table 28'], axis = 1)

#Rename the columns
police.columns = ['County', 'Total Officers', 'Officers', 'Civilian Officers']

#Extract certain columns from the dataset
police = police[['County', 'Total Officers']]
police.head()

,County,Total Officers
0,Alameda,"1,564"
1,Butte,269
2,Contra Costa,957
3,El Dorado,349
4,Fresno,"1,104"


<b>Income Per County</b>

In [4]:
# Import the Income Dataset
income = pd.read_csv("Datasets/Income Per Capita/Income Per Capita 2016.csv")

# Rename the columns to the real names
income = income.rename(columns=income.iloc[0, :])

# Remove the old columns row
income = income.iloc[1:, :]

# Remove the California Summary Row
income = income.drop(income.index[40])

# Retrieve only the column containing the mean income of households
income = income[["Geographic Area Name",
                 "Mean income (dollars)!!Estimate!!PER CAPITA INCOME BY RACE AND HISPANIC OR LATINO ORIGIN!!Total population",
                 "Mean income (dollars)!!Margin of Error!!PER CAPITA INCOME BY RACE AND HISPANIC OR LATINO ORIGIN!!Total population"]]

# Rename the columns to be more readable
income.columns = ["County", "Mean Income Per Capita", "Mean Income Per Capita Margin of Error"]

# Reindex the rows
income = income.reset_index(drop=True)

# Print the Dataset and Summary
income.head()

,County,Mean Income Per Capita,Mean Income Per Capita Margin of Error
0,"Alameda County, California",42376,720
1,"Butte County, California",25938,1552
2,"Contra Costa County, California",43120,871
3,"El Dorado County, California",39284,3087
4,"Fresno County, California",22518,603


**Population Density**

No Cleaning Needed!

**Crime**

In [26]:
# Import the Crime Dataset
crime = pd.read_csv("Datasets/Crimes Per County/Crimes_and_Clearances.csv")

# Retriving Data containing Years 2010-2019 
#crime = crime[crime['Year'].astype('int').isin.contains('2016')]
#crime = crime['Year'].isin(['2016'])

crime = crime[(crime['Year'] >= 2010) & (crime['Year'] <= 2019)]

# Lower Case the County Names
crime['County'] = crime['County'].str.lower()

# Stripping 'County' from County Names
def county(county_name):
    return county_name.strip('county')

# Returns the Stripped County Names
crime['County'] = crime['County'].apply(county)

# Sum Crime Columns
crime_sum = crime['Violent_sum'] + crime['Homicide_sum'] + crime['ForRape_sum'] + crime['Robbery_sum'] + crime['AggAssault_sum'] + crime['Property_sum'] + crime['Burglary_sum'] +  crime['VehicleTheft_sum'] + crime['LTtotal_sum']
crime['Sum of Crimes'] = crime_sum

# Drop
crime = crime.drop(columns=['NCICCode', 'Violent_sum', 'Homicide_sum', 'ForRape_sum',
                            'Robbery_sum', 'AggAssault_sum','Property_sum', 'Burglary_sum',
                            'VehicleTheft_sum', 'LTtotal_sum', 'ViolentClr_sum', 'HomicideClr_sum',
                            'ForRapeClr_sum', 'RobberyClr_sum', 'AggAssaultClr_sum', 'PropertyClr_sum',
                            'BurglaryClr_sum', 'VehicleTheftClr_sum', 'LTtotalClr_sum', 'TotalStructural_sum',
                            'TotalMobile_sum', 'TotalOther_sum', 'GrandTotal_sum', 'GrandTotClr_sum',
                            'RAPact_sum', 'ARAPact_sum', 'FROBact_sum', 'KROBact_sum', 'OROBact_sum',
                            'SROBact_sum', 'HROBnao_sum', 'CHROBnao_sum', 'GROBnao_sum', 'CROBnao_sum',
                            'RROBnao_sum', 'BROBnao_sum', 'MROBnao_sum', 'FASSact_sum', 'KASSact_sum', 
                            'OASSact_sum', 'HASSact_sum', 'FEBURact_Sum', 'UBURact_sum', 'RESDBUR_sum',
                            'RNBURnao_sum', 'RDBURnao_sum', 'RUBURnao_sum', 'NRESBUR_sum', 'NNBURnao_sum',
                            'NDBURnao_sum', 'NUBURnao_sum', 'MVTact_sum', 'TMVTact_sum', 'OMVTact_sum',
                            'PPLARnao_sum', 'PSLARnao_sum', 'PSLARnao_sum', 'SLLARnao_sum', 'MVLARnao_sum',
                            'MVPLARnao_sum', 'BILARnao_sum', 'FBLARnao_sum', 'COMLARnao_sum', 'AOLARnao_sum',
                            'LT400nao_sum', 'LT200400nao_sum', 'LT50200nao_sum', 'LT50nao_sum'])

# Print Arrests Dataset
crime.head()

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Year,County,Sum of Crimes
834,2010,fresno,436
835,2011,fresno,394
836,2012,fresno,428
837,2013,fresno,484
838,2014,fresno,296
